In [4]:
# !pip install requests

In [5]:
import requests
import re
import time
import json

In [ ]:
def limpar_cnpj(cnpj):
	"""Remove todos os caracteres não numéricos de um CNPJ."""
	return re.sub(r'[^0-9]', '', cnpj)

def consultar_cnpjs(lista_cnpjs):
	"""Consulta uma lista de CNPJs na Open CNPJ API."""
	
	base_url = "https://open.cnpja.com/office/"
	
	resultados = {}
	
	for cnpj_formatado in lista_cnpjs:
		
		cnpj_limpo = limpar_cnpj(cnpj_formatado)

		url_consulta = f"{base_url}{cnpj_limpo}"
		
		print(f"\n*** CNPJ: {cnpj_formatado}")
		
		try:
			# Requisição GET ---- 'timeout' para não ficar esperando pra sempre
			headers = {
				"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36",
				"Accept": "application/json"
			}
			response = requests.get(url_consulta, headers=headers, timeout=10)
			
			# * Código 200 - Consulta bem-sucedida
			if response.status_code == 200:
				dados_empresa = response.json()
				resultados[cnpj_formatado] = dados_empresa
				
				print(dados_empresa)

			# Erros esperados da API
			# 404 - CNPJ não encontrado
			elif response.status_code == 404:
				print(f"Erro: CNPJ '{cnpj_formatado}' não encontrado.")
				resultados[cnpj_formatado] = {"erro": "CNPJ não encontrado"}
			# Outros erros
			else:
				print(f"Erro ao consultar: Status Code {response.status_code}")
				resultados[cnpj_formatado] = {"erro": f"Status {response.status_code}"}
		
		# Erros de conexão (sem internet, timeout, etc.)
		except requests.exceptions.RequestException as e:
			
			print(f"Erro de conexão: {e}")
			resultados[cnpj_formatado] = {"erro": f"Erro de conexão: {e}"}
		
		print("\n" + "#" * 40 + "\n")
		
		# Uma pequena pausa é uma boa prática para não sobrecarregar a API
		time.sleep(1) 
		
	return resultados


# Lista de CNPJs para consulta
meus_cnpjs = [
	"45.954.282/0001-02",   # CNPJ da própria Principia Educação
	"00.000.000/0001-91",   # Banco do Brasil (Exemplo Matriz)
	"19.131.243/0001-97",   # Open Knowledge Brasil
	"99999999000191",       # Exemplo de CNPJ inválido
]

dados_das_empresas = consultar_cnpjs(meus_cnpjs)

# Tranformar em JSON
dados_em_json = json.dumps(dados_das_empresas, indent=4, ensure_ascii=False)
with open("data/OpenCNPJA_dados_empresas.json", "w", encoding="utf-8") as arquivo:
	arquivo.write(dados_em_json)



*** CNPJ: 45.954.282/0001-02
{'updated': '2025-10-11T00:00:00.000Z', 'taxId': '45954282000102', 'alias': 'Principia', 'founded': '2022-04-07', 'head': True, 'company': {'members': [{'since': '2022-11-17', 'person': {'id': '09cb90d1-7aa5-4a70-b2e6-99b76285d2ca', 'type': 'LEGAL', 'name': 'PRINCIPIA EDUCATION TECHNOLOGY LLC', 'taxId': '48249302000124'}, 'role': {'id': 37, 'text': 'Sócio Pessoa Jurídica Domiciliado no Exterior'}, 'agent': {'person': {'id': '1a115663-ccf3-4d8b-a775-430d5cacb7dc', 'type': 'NATURAL', 'name': 'Newton Maia Salomao Alves', 'taxId': '***658924**', 'age': '41-50'}, 'role': {'id': 17, 'text': 'Procurador'}}}, {'since': '2022-04-07', 'person': {'id': '1a115663-ccf3-4d8b-a775-430d5cacb7dc', 'type': 'NATURAL', 'name': 'Newton Maia Salomao Alves', 'taxId': '***658924**', 'age': '41-50'}, 'role': {'id': 5, 'text': 'Administrador'}}], 'id': 45954282, 'name': 'PRINCIPIA EDUCACAO TECNOLOGIA E SERVICOS LTDA', 'equity': 147264212, 'nature': {'id': 2062, 'text': 'Sociedade E